In [17]:
import os
import re
import time
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import fitz

In [18]:
df_pt = pd.read_csv("../../dataset/pt/atas_dataset.csv")
df_en = pd.DataFrame(columns=df_pt.columns)
df_en = pd.concat([df_en, df_pt[['data', 'selic', 'decisao']]], ignore_index=True)

C:\Users\ryanf\AppData\Local\Temp\ipykernel_16212\4163163525.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_en = pd.concat([df_en, df_pt[['data', 'selic', 'decisao']]], ignore_index=True)


In [19]:
print(df_en)

         data texto_ata  pdf decisao  selic
0    20250319       NaN  NaN       1  14.25
1    20250129       NaN  NaN       1  13.15
2    20241211       NaN  NaN       1  12.15
3    20241106       NaN  NaN       1  11.15
4    20240918       NaN  NaN       1  10.65
..        ...       ...  ...     ...    ...
222  20000524       NaN  NaN      -1  18.39
223  20000419       NaN  NaN      -1  18.55
224  20000322       NaN  NaN       1  18.94
225  20000216       NaN  NaN       1  18.88
226  20000119       NaN  NaN      -1  18.87

[227 rows x 5 columns]


In [20]:
# === Configurações iniciais ===
BASE_URL = "https://www.bcb.gov.br"
CRONOLOGICOS_URL = f"{BASE_URL}/en/publications/copomminutes/cronologicos"
PDF_DIR = "../pdf_en"

In [21]:
# Cria a pasta pdf se não existir
os.makedirs(PDF_DIR, exist_ok=True)

In [43]:
try:
    url = "https://www.bcb.gov.br/en/publications/copomminutes/cronologicos"
    options = Options()
    options.headless = True   
    # == configurando env == 
    service = Service(ChromeDriverManager().install())
    driver= webdriver.Chrome(options=options)
    driver.get(url)
    WebDriverWait(driver, 15).until(
       EC.presence_of_element_located((By.XPATH, "//a[.//span[text()='file_download']]"))  # ou outro seletor adequado
    )
    time.sleep(5)
    
    resultados_div = driver.find_element(By.CLASS_NAME, "resultados-relacionados.row")
    links = resultados_div.find_elements(By.TAG_NAME, "a")
    hrefs = [link.get_attribute("href") for link in links if link.get_attribute("href")]
    
    print(len(hrefs))
    
except Exception as e:
    print(f"Erro ao ler {url}: {e}")
finally:
    driver.quit()
    

Erro ao ler https://www.bcb.gov.br/en/publications/copomminutes/cronologicos: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7615CA145+76773]
	GetHandleVerifier [0x00007FF7615CA1A0+76864]
	(No symbol) [0x00007FF761388F7A]
	(No symbol) [0x00007FF7613DF496]
	(No symbol) [0x00007FF7613DF74C]
	(No symbol) [0x00007FF761432287]
	(No symbol) [0x00007FF76140739F]
	(No symbol) [0x00007FF76142F0CF]
	(No symbol) [0x00007FF761407133]
	(No symbol) [0x00007FF7613D04D1]
	(No symbol) [0x00007FF7613D1263]
	GetHandleVerifier [0x00007FF76188A8ED+2962317]
	GetHandleVerifier [0x00007FF761884EC2+2939234]
	GetHandleVerifier [0x00007FF7618A2FF3+3062419]
	GetHandleVerifier [0x00007FF7615E4B9A+185914]
	GetHandleVerifier [0x00007FF7615EC78F+217647]
	GetHandleVerifier [0x00007FF7615D2A44+111844]
	GetHandleVerifier [0x00007FF7615D2BF2+112274]
	GetHandleVerifier [0x00007FF7615B8A79+5401]
	BaseThreadInitThunk [0x00007FFE5A537374+20]
	RtlUserThreadStart [0x00007FFE5B39CC91+33]



In [56]:
#Por scrapping apenas alguns arquivos foram baixados, por falta de padronização no site do bcb, com isso o restante dos arquivos foram baixados manualmente e comparados com os dados em português para garantir que continham todos os dados
arquivos = os.listdir("../../dataset/pdf_en")
for data in df_en['data']:
    arquivo = 'ata_'+ str(data) + '.pdf'
    if arquivo not in arquivos:
        print(f"arquivo não encontrado {arquivo}")

for arquivo in arquivos:
    data_arquivo = re.search(r'(\d{8})', arquivo).group(1)
    if data_arquivo not in df_en['data'].astype(str).values:
        print(f"Existe o arquivo {arquivo} que não tem data {data} no df")

In [57]:
for arquivo in os.listdir("../../dataset/pdf_en"):
    data_arquivo = re.search(r'(\d{8})', arquivo).group(1)
    if data_arquivo in df_en['data'].astype(str).values:
        caminho_pdf = os.path.join(PDF_DIR, arquivo)
        try:
            print(f"Lendo {arquivo}...")
            doc = fitz.open(caminho_pdf)
            texto = ""
            for pagina in doc:
                texto += pagina.get_text()
            doc.close()

            df_en.loc[df_en['data'].astype(str) == data_arquivo, ['texto_ata', 'pdf']] = [texto, 1]
            print(f"arquivo {arquivo} atualizou o dataset {data}")


        except Exception as e:
            print(f"Erro ao processar {arquivo}: {e}")
    else:
        print(f"Arquivo {arquivo} não compativel com datas do dataset")

Lendo ata_20000119.pdf...
arquivo ata_20000119.pdf atualizou o dataset 20000119
Arquivo ata_20000119.pdf não compativel com datas do dataset
Lendo ata_20000216.pdf...
arquivo ata_20000216.pdf atualizou o dataset 20000119
Arquivo ata_20000216.pdf não compativel com datas do dataset
Lendo ata_20000322.pdf...
arquivo ata_20000322.pdf atualizou o dataset 20000119
Arquivo ata_20000322.pdf não compativel com datas do dataset
Lendo ata_20000419.pdf...
arquivo ata_20000419.pdf atualizou o dataset 20000119
Arquivo ata_20000419.pdf não compativel com datas do dataset
Lendo ata_20000524.pdf...
arquivo ata_20000524.pdf atualizou o dataset 20000119
Arquivo ata_20000524.pdf não compativel com datas do dataset
Lendo ata_20000620.pdf...
arquivo ata_20000620.pdf atualizou o dataset 20000119
Arquivo ata_20000620.pdf não compativel com datas do dataset
Lendo ata_20000719.pdf...
arquivo ata_20000719.pdf atualizou o dataset 20000119
Arquivo ata_20000719.pdf não compativel com datas do dataset
Lendo ata_200

In [60]:
df_en.isnull().values.any()

False

In [61]:
# Salvar em CSV
df_en.to_csv("../../dataset/en/atas_copom_ingles.csv", index=False, encoding="utf-8")